In [4]:
import numpy as np
import time
import open3d
import pandas as pd
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [1]:
def pcl_pipeline(pcd):
    pcd = open3d.io.read_point_cloud(pcd)
    pcd_down = pcd.voxel_down_sample(voxel_size = 0.25)
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.30,
                                         ransac_n=3,
                                             num_iterations=150)
    [a, b, c, d] = plane_model
    
    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([0, 1.0, 1.0])
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    outlier_cloud.paint_uniform_color([1.0, 0, 0])
    with open3d.utility.VerbosityContextManager(open3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.45, min_points=7, print_progress=True))

    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    outlier_cloud.colors = open3d.utility.Vector3dVector(colors[:, :3])
    obbs = []
    indexes = pd.Series(range(len(labels))).groupby(labels,sort = False).apply(list).tolist()

    max_points = 300
    min_points = 40

    for i in range(0,len(indexes)):
        nb_points = len(outlier_cloud.select_by_index(indexes[i]).points)
        if (nb_points > min_points and nb_points < max_points):
            sub_cloud = outlier_cloud.select_by_index(indexes[i])
            obb = sub_cloud.get_axis_aligned_bounding_box()
            obb.color = (0,0,1)
            obbs.append(obb)
    
    list_of_visuals = []
    list_of_visuals.append(outlier_cloud)
    list_of_visuals.extend(obbs)
    list_of_visuals.append(inlier_cloud)
    return list_of_visuals

In [2]:
import glob
import cv2
video_points = sorted(glob.glob("test_files/KITTI/*.pcd"))

In [9]:
# video_points = sorted(glob.glob("test_files/KITTI/*.pcd"))
vis = open3d.visualization.Visualizer()
vis.create_window()

for idx, file2 in enumerate(video_points):
    list_of_visuals = pcl_pipeline(file2)
    vis.add_geometry(list_of_visuals)
    vis.update_geometry(list_of_visuals)
    vis.update_renderer()
    vis.poll_events()
    vis.update_renderer()
    vis.remove_geometry(list_of_visuals)
vis.destroy_window()

1
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 137
2


TypeError: add_geometry(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.visualization.Visualizer, geometry: open3d.cpu.pybind.geometry.Geometry, reset_bounding_box: bool = True) -> bool

Invoked with: Visualizer with name Open3D, [PointCloud with 59292 points., AxisAlignedBoundingBox: min: (54.738, -5.231, -0.584), max: (56.548, 3.215, 2.089), AxisAlignedBoundingBox: min: (55.199, 0.748, 0.119), max: (56.623, 2.878, 2.107), AxisAlignedBoundingBox: min: (13.36, 18.525, 0.445), max: (14.486, 19.737, 1.013), AxisAlignedBoundingBox: min: (11.454, 19.385, 0.439), max: (12.968, 19.929, 0.998), AxisAlignedBoundingBox: min: (-2.932, 14.152, 0.535), max: (1.194, 14.92, 0.723), AxisAlignedBoundingBox: min: (-18.643, 16.603, 0.285), max: (-17.116, 17.019, 1.046), AxisAlignedBoundingBox: min: (-22.641, 18.951, -0.023), max: (-21.092, 19.214, 1.211), AxisAlignedBoundingBox: min: (-33.912, 20.493, -0.916), max: (-24.397, 21.232, 1.553),

In [7]:
result_video = []
for ids, pts in enumerate(video_points):
    print(ids)
    pc = open3d.io.read_point_cloud(video_points[ids])
    result_video.append(pcl_pipeline(pc))
    

0
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 141
1
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 223
2
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 301
3
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 58
4
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 253
5
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 172
6
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute n

In [8]:
open3d.visualization.draw_geometries(result_video[0])

In [9]:
out = cv2.VideoWriter("output.avi",cv2.VideoWriter_fourcc(*"DIVX"),15,(96,96))
for i in range(len(result_video)):
    out.write(result_video[i])
    
out.release()

TypeError: Expected Ptr<cv::UMat> for argument 'image'